In [1]:
from matplotlib import pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

shapes = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', '+']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'darkgreen', 'chocolate']

In [2]:
DRIs = dict()
VRIs = dict()
VPlusDs = dict()
VMulDs = dict()
DCs = dict()
BCs = dict()
CCs = dict()
PGs = dict()
ERs = dict()
GCs = dict()
GCDs = dict()
GGCs = dict()
GGCDs = dict()
iFits = dict()

AFOs = dict()
AFPs = dict()

import os

from experiment.functions import func_get_lab_graph
from experiment.lab2._0_config_ import SoftwareConfigs1, LabSpace

Graphs = dict()
Labels = dict()
Types = dict()


for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')

        if not os.path.isdir(labDir):
            os.makedirs(labDir, exist_ok=True)

        G, labelDict, _, typeDict = func_get_lab_graph(modelDir, labDir, softwareName)
        Graphs[softwareName] = G
        Labels[softwareName] = labelDict
        Types[softwareName] = typeDict
        
        DRIs[softwareName] = dict(G.nodes.data('DRI'))
        VRIs[softwareName] = dict(G.nodes.data('VRI'))
        VPlusDs[softwareName] = dict(G.nodes.data('VPlusD'))
        VMulDs[softwareName] = dict(G.nodes.data('VMulD'))
        
        DCs[softwareName] = dict(G.nodes.data('dc'))
        BCs[softwareName] = dict(G.nodes.data('bc'))
        CCs[softwareName] = dict(G.nodes.data('cc'))
        PGs[softwareName] = dict(G.nodes.data('pg'))
        ERs[softwareName] = dict(G.nodes.data('er'))
        GCs[softwareName] = dict(G.nodes.data('gc'))
        GCDs[softwareName] = dict(G.nodes.data('gcd'))
        GGCs[softwareName] = dict(G.nodes.data('ggc'))
        GGCDs[softwareName] = dict(G.nodes.data('ggcd'))
        iFits[softwareName] = dict(G.nodes.data('iFit'))
        
        AFOs[softwareName] = dict(G.nodes.data('AFO'))
        AFPs[softwareName] = dict(G.nodes.data('AFP'))

print('Extract data for risk diffusion experiments')

# SI Infectious Disease Model

## Parameter settings

In [3]:
TestMethods = [
    ('DRI', DRIs),
    ('VRI', VRIs),
    ('DC', DCs),
    ('BC', BCs),
    ('VPlusD', VPlusDs),
    ('VMulD', VMulDs),
    # ('CC', CCs),
    # ('PageRank', PGs),
    # ('ElementRank', ERs),
    ('GC', GCs),
    ('$GC_d$', GCDs),
    ('GGC', GGCs),
    ('$GGC_d$', GGCDs),
    ('iFit', iFits),
]

Thresholds = dict()
Influences = dict()

for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName]
        afos = AFOs[softwareName]
        afps = AFPs[softwareName]
        
        influences = dict()
        for (u, v) in G.edges():
            influences[(u, v)] = afos[u] * afps[v]
        Influences[softwareName] = influences
        # Influences[softwareName] = func_get_edge_influence(G, 'w-in-degree')
        Thresholds[softwareName] = {node: 0.5 for node in G.nodes}
        pass

SISimDir = os.path.join(LabSpace, '.sim.SI')
if not os.path.isdir(SISimDir):
    os.mkdir(SISimDir)


## simulation experiment

### Calculate network efficiency

In [4]:
import copy
from experiment.algorithms.utils import cal_network_efficiency, cal_network_complexity
from experiment.functions import func_SortDictByValue
from experiment.algorithms.propagation_models.sir_epidemic import sir_epidemic
import pandas as pd

TestMethods_ = ['DRI', 'VRI', 'VPlusD', 'VMul'
                                        'D', 'DC', 'BC', 'GC', 'GGC', '$GC_d$', '$GGC_d$', 'iFit']
cols = ['name'] + TestMethods_
dfEfficiency = pd.DataFrame(columns=cols)
dfComplexity = pd.DataFrame(columns=cols)


+Steps = 100

SimThreshold = 0.05

SimCount = 100

idx = 0
for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName] 
        labels = Labels[softwareName]
        types = Types[softwareName]
        N = G.number_of_nodes()
        
        efficiencyDict = {
            'name': softwareName
        }
        complexityDict = {
            'name': softwareName
        }
        
        srcEfficiency = cal_network_efficiency(G)
        srcComplexity = cal_network_complexity(G)
        
        if isinstance(SimThreshold, float) and 0 <= SimThreshold <= 1:
            seedsCount = int(G.number_of_nodes() * SimThreshold)
        elif isinstance(SimThreshold, int) and 0 < SimThreshold < N:
            seedsCount = SimThreshold
        else:
            seedsCount = int(N * 0.15)
        
        print(f'处理软件 {softwareName}')
        for methodName, methods in TestMethods:
            if methodName not in TestMethods_:
                continue
                
            method = methods[softwareName]
            SortedMethod = func_SortDictByValue(method)
            SortedMethodIds = list(SortedMethod.keys())
            seeds = SortedMethodIds[:seedsCount]
            print(f'\t处理方法 {methodName}')
            
            totalEfficiency = 0.0
            totalComplexity = 0.0
            for i in range(SimCount):
                simNodes, simRoundCounts, simRoundNodes = sir_epidemic(G, seeds=seeds, step=Steps, beta=0.1, gamma=0.0) 
                tG = copy.deepcopy(G)
                tG.remove_nodes_from(simNodes)
                tEfficiency = cal_network_efficiency(tG)
                totalEfficiency += tEfficiency
                tComplexity = cal_network_complexity(tG)
                totalComplexity += tComplexity
            totalEfficiency /= SimCount
            totalComplexity /= SimCount
            
            efficiencyDict[methodName] = round((srcEfficiency - totalEfficiency) / srcEfficiency, 3)
            complexityDict[methodName] = round((srcComplexity - totalComplexity) / srcComplexity, 3)
        
        dfEfficiency.loc[idx] = efficiencyDict
        dfComplexity.loc[idx] = complexityDict
        idx += 1
dfEfficiency.to_csv(os.path.join(SISimDir, 'Efficiency.csv'))
dfComplexity.to_csv(os.path.join(SISimDir, 'Complexity.csv'))


### Draw the diffusion degree of SI models at different scales

In [4]:
import shutil

saveDir = os.path.join(LabSpace, 'graph', 'DRI-SI')
if os.path.isdir(saveDir):
    shutil.rmtree(saveDir)
os.makedirs(saveDir)    

In [6]:
from experiment.functions import func_SortDictByValue
from experiment.algorithms.propagation_models.sir_epidemic import sir_epidemic
import pandas as pd

TestMethods_ = ['DRI', 'DC', 'BC', 'GC', 'GGC', '$GC_d$', '$GGC_d$', 'iFit']
labs = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

cols = ['name'] + TestMethods_

Steps = 100

SimThreshold = 0.05

SimCount = 500

fig, axs = plt.subplots(3, 3, figsize=(18, 15))
plt.subplots_adjust(hspace=0.3, wspace=0.22)
idx = 0
maxDict = dict()
for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName] 
        labels = Labels[softwareName]
        types = Types[softwareName]
        N = G.number_of_nodes()
        maxDict[softwareName] = dict()
        
        if isinstance(SimThreshold, float) and 0 <= SimThreshold <= 1:
            seedsCount = int(G.number_of_nodes() * SimThreshold)
        elif isinstance(SimThreshold, int) and 0 < SimThreshold < N:
            seedsCount = SimThreshold
        else:
            seedsCount = int(N * 0.15)
            
        print(f'{softwareName} starts SI simulation, with an initial number of nodes {seedCount}')
        
        ax = axs[int(idx / 3), idx % 3]
        # ax = axs[idx]
        allSiSimCounts = dict()
        countDf = pd.DataFrame()
        
        mI = 0
        for methodName, methods in TestMethods:
            if methodName not in TestMethods_:
                continue
                
            print(f'Simulate method {methodName} {SimCount} times')
            method = methods[softwareName]
            
            SortedMethod = func_SortDictByValue(method)
            SortedMethodIds = list(SortedMethod.keys())
            seeds = SortedMethodIds[:seedsCount]
            
            influences = Influences[softwareName]
            thresholds = Thresholds[softwareName]
            
            roundDF = pd.DataFrame(
               columns = [f'round {i}' for i in range(SimCount)]
            )
            
            # ks = sum([G.degree(n) for n in G])
            # k2s = sum([G.degree(n)**2 for n in G])
            
            for i in range(SimCount):
                simNodes, simRoundCounts, simRoundNodes = sir_epidemic(G, seeds=seeds, step=Steps, beta=0.1, gamma=0.0)
                # simNodes, simRoundCounts, simRoundNodes = sir_epidemic(G, seeds=seeds, step=Steps, influence=influences)
                siSimCounts = [0]
                siSimCounts.extend(simRoundCounts)  
                roundDF[f'round {i}'] = siSimCounts
                        
            resDf = roundDF.T
            countDf[methodName] = resDf.mean()
            ax.plot(countDf[methodName], label=f'{methodName}={round(max(resDf.mean()), 2)}', color=colors[mI])
            maxDict[softwareName][methodName] = max(resDf.mean())
            mI += 1
        
        countDf.to_csv(os.path.join(SISimDir, f'{softwareName}.sim.csv'))
        countDf.to_csv(os.path.join(saveDir, f'{softwareName}.csv'))
        
        ax.set_xlabel('Iterations', fontdict={'size': 12})
        ax.set_ylabel('Number of infected nodes', fontdict={'size': 12})
        ax.set_title(f'{labs[idx]}) {softwareName}', y=-0.25, fontdict={'family': 'Times New Roman', 'size': 18})
        ax.legend(loc='best')
        
        idx += 1